## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-13-10-23-11 +0000,bbc,More rules being considered to keep children s...,https://www.bbc.com/news/articles/cp82447l84ko
1,2025-07-13-10-21-32 +0000,bbc,The summer holiday hacks that saved us hundred...,https://www.bbc.com/news/articles/c4g84nrlvv7o
2,2025-07-13-09-24-38 +0000,bbc,Health secretary and BMA to meet in bid to avo...,https://www.bbc.com/news/articles/c4g8xq0e928o
3,2025-07-13-09-00-31 +0000,nyt,"In Canada’s Northern Outposts, Rusting Relics ...",https://www.nytimes.com/2025/07/13/world/ameri...
4,2025-07-13-07-17-55 +0000,bbc,US drops charges against doctor accused of des...,https://www.bbc.com/news/articles/cy0w1p0wq87o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
44,trump,5
3,children,2
46,tariff,2
42,mexico,2
26,against,2


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
13,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...,24
8,2025-07-13-01-00-00 +0000,wsj,Economists expect stronger growth and job crea...,https://www.wsj.com/economy/economists-see-low...,23
7,2025-07-13-01-04-00 +0000,wsj,Answers to frequently asked questions about Th...,https://www.wsj.com/economy/economic-forecasti...,19
14,2025-07-12-14-00-00 +0000,wsj,The White House has an explanation for the nea...,https://www.wsj.com/politics/policy/trump-new-...,18
9,2025-07-12-23-48-24 +0000,bbc,Trump's tariffs are looming large over the UK’...,https://www.bbc.com/news/articles/c5yp5qzeer6o,12


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
13,24,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...
8,23,2025-07-13-01-00-00 +0000,wsj,Economists expect stronger growth and job crea...,https://www.wsj.com/economy/economists-see-low...
14,13,2025-07-12-14-00-00 +0000,wsj,The White House has an explanation for the nea...,https://www.wsj.com/politics/policy/trump-new-...
3,11,2025-07-13-09-00-31 +0000,nyt,"In Canada’s Northern Outposts, Rusting Relics ...",https://www.nytimes.com/2025/07/13/world/ameri...
7,11,2025-07-13-01-04-00 +0000,wsj,Answers to frequently asked questions about Th...,https://www.wsj.com/economy/economic-forecasti...
2,8,2025-07-13-09-24-38 +0000,bbc,Health secretary and BMA to meet in bid to avo...,https://www.bbc.com/news/articles/c4g8xq0e928o
11,8,2025-07-12-19-46-50 +0000,bbc,Rapidly growing fire near Grand Canyon prompts...,https://www.bbc.com/news/articles/czjk10wwmkeo
0,7,2025-07-13-10-23-11 +0000,bbc,More rules being considered to keep children s...,https://www.bbc.com/news/articles/cp82447l84ko
4,7,2025-07-13-07-17-55 +0000,bbc,US drops charges against doctor accused of des...,https://www.bbc.com/news/articles/cy0w1p0wq87o
9,7,2025-07-12-23-48-24 +0000,bbc,Trump's tariffs are looming large over the UK’...,https://www.bbc.com/news/articles/c5yp5qzeer6o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
